In [3]:
import sys
sys.path.append('..')

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from util.preprocessing import parse_num_of_people, extract_month, extract_facility, extract_ratio, extract_population, extract_cost, calc_damage_scale, extract_middle_class, str_to_median, count_days
from util.preprocessing2 import extract_sago#, extract_date, is_noramal_temper, is_normal_humid, get_temper, get_humid
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import OneSidedSelection, EditedNearestNeighbours
import re
from dateutil import parser

def preprocess_data():
    df = pd.read_csv('../data/output-v2.csv')
    #필요 없는 특성 제거

    allColumns = df.columns
    params = ['발생일시','공공/민간 구분', '기상상태', '시설물 종류', '사망자수(명)', '부상자수(명)', '공사비', '공사기간', '공정률', '작업자수', '설계안전성검토', '공종', '사고신고사유']

    df = df.drop(allColumns.drop(params), axis=1)

    df = df.dropna()

    #범주형 데이터를 수치형 데이터로 인코딩
    df['공종'] = df['공종'].apply(extract_middle_class)
    df['사망자수(명)'] = df['사망자수(명)'].apply(parse_num_of_people)
    df['부상자수(명)'] = df['부상자수(명)'].apply(parse_num_of_people)
    #df['사고발생월'] = df['발생일시'].apply(extract_month)
    df['시설물 종류'] = df['시설물 종류'].apply(extract_facility)
    df['공사기간'] = df['공사기간'].apply(count_days)
    df['공정률'] = df['공정률'].apply(str_to_median)
    df['작업자수'] = df['작업자수'].apply(extract_population)
    df['공사비'] = df['공사비'].apply(extract_cost)
    df['피해규모'] = df.apply(calc_damage_scale, axis=1)
    df['발생일시'] = df['발생일시'].apply(extract_month)
    
    # 기상 상태 컬럼 분리
    df[['날씨', '기온', '습도']] = df['기상상태'].str.extract('날씨 : (\S+)기온 : (\d+)℃습도 : (\d+)%')
    df = df.drop(['기상상태'], axis=1)
    '''
    # SOYEON
    # 기온, 습도 결측치 처리
    df[['기온', '습도']].fillna(-999)
    df[['기온', '습도']] = df[['기온', '습도']].astype('float64')
    df['습도'] = df['습도'].apply(lambda x: x/100)
    if df['기온'].apply(is_normal_temper) == False: 
        df['기온'] = df['발생일시'].apply(get_temper)
    if df['습도'].apply(is_normal_humid) == False:
        df['습도'] = df['발생일시'].apply(get_humid)
    '''
    # SOYEON
    # 사고신고사유 열 추가
    df['사망사고', '부상사고', '재산피해', '기타'] = df['사고신고사유'].apply(extract_sago)
    df = df.drop(['사고신고사유'], axis=1)

    
    print(df)
    
preprocess_data()

       발생일시 공공/민간 구분   시설물 종류        공종  사망자수(명)  부상자수(명)           공사비   
0         4       민간      건축   건축물 부대공사      0.0      1.0  1.500000e+09  \
1         4       민간      건축   건축물 부대공사      0.0      1.0  7.500000e+09   
2         4       공공      토목        관공사      0.0      1.0  1.750000e+10   
3         4       공공      건축   철근콘크리트공사      0.0      1.0  1.250000e+10   
4         4       공공  산업환경설비     기계설비공사      0.0      1.0           NaN   
...     ...      ...      ...       ...      ...      ...           ...   
17788     7       민간      건축     전기설비공사      0.0      1.0  1.500000e+09   
17789     7       공공      토목       가설공사      0.0      1.0  7.500000e+08   
17790     7       민간      건축       가설공사      0.0      1.0  1.750000e+10   
17791     7       민간      건축         기타      0.0      1.0  1.500000e+09   
17792     1       민간      건축       가설공사      0.0      1.0  3.500000e+09   

         공사기간    공정률  작업자수 설계안전성검토  피해규모  날씨   기온  습도 (사망사고, 부상사고, 재산피해, 기타)  
0      1171.0  0.950